<a href="https://colab.research.google.com/github/deepavasanthkumar/deepcodesnippets/blob/master/Spark_Generating_Incremental_Numbers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import pyspark
from pyspark.sql import SparkSession
  
spark = SparkSession.builder.appName("Spark Window Functions ").getOrCreate()
spark

In [12]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
data1 = [("James","","Smith","36636","M", 1000, "Sales", 2020),
    ("Michael","Rose","","40288","M", 2000, "Operations",2020),
    ("Robert","","Williams","42114","M", 3000, "Sales",2020),
    ("Maria","Anne","Jones","39192","F", 4000, "Operations",2020),
  ("Ria","Anne","Jones","60000","F", 7000, "Operations",2020)
  
  ]
 
schema1 = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True),
    StructField("annualsalary", IntegerType(), True),
    StructField("work", StringType(), True),
    StructField("year", IntegerType(), True),
   
  ])

df1 = spark.createDataFrame(data=data1,schema=schema1)
df1.show(truncate=False)

+---------+----------+--------+-----+------+------------+----------+----+
|firstname|middlename|lastname|id   |gender|annualsalary|work      |year|
+---------+----------+--------+-----+------+------------+----------+----+
|James    |          |Smith   |36636|M     |1000        |Sales     |2020|
|Michael  |Rose      |        |40288|M     |2000        |Operations|2020|
|Robert   |          |Williams|42114|M     |3000        |Sales     |2020|
|Maria    |Anne      |Jones   |39192|F     |4000        |Operations|2020|
|Ria      |Anne      |Jones   |60000|F     |7000        |Operations|2020|
+---------+----------+--------+-----+------+------------+----------+----+



#monotonically_increasing_id



monotonically_increasing_id is guaranteed to be monotonically increasing and unique, but **not consecutive**.

The current implementation puts the partition ID in the upper 31 bits, and the record number within each partition in the lower 33 bits. 
 The assumption is that the data frame has less than 1 billion partitions, and each partition has less than 8 billion records.
 



In [13]:
from pyspark.sql.functions import monotonically_increasing_id
df1.select(monotonically_increasing_id().alias('counter')).collect()

[Row(counter=0),
 Row(counter=1),
 Row(counter=8589934592),
 Row(counter=8589934593),
 Row(counter=8589934594)]

As an example, consider a DataFrame with two partitions, each with 2 & 3 records. This expression would return the following IDs: 0, 1, 8589934592 (1L << 33), 8589934593, 8589934594.

In [14]:
df2 = df1.withColumn("counter", monotonically_increasing_id())
df2.show(truncate=False)

+---------+----------+--------+-----+------+------------+----------+----+----------+
|firstname|middlename|lastname|id   |gender|annualsalary|work      |year|counter   |
+---------+----------+--------+-----+------+------------+----------+----+----------+
|James    |          |Smith   |36636|M     |1000        |Sales     |2020|0         |
|Michael  |Rose      |        |40288|M     |2000        |Operations|2020|1         |
|Robert   |          |Williams|42114|M     |3000        |Sales     |2020|8589934592|
|Maria    |Anne      |Jones   |39192|F     |4000        |Operations|2020|8589934593|
|Ria      |Anne      |Jones   |60000|F     |7000        |Operations|2020|8589934594|
+---------+----------+--------+-----+------+------------+----------+----+----------+



#row_number

pyspark.sql.functions.row_number()
Window function: returns a sequential number starting at 1 within a window partition.

To use row_number() the data needs to be **sortable**.


In [15]:
df1.createOrReplaceTempView('df1')
spark.sql('select row_number() over (order by id) as num, * from df1').show()

+---+---------+----------+--------+-----+------+------------+----------+----+
|num|firstname|middlename|lastname|   id|gender|annualsalary|      work|year|
+---+---------+----------+--------+-----+------+------------+----------+----+
|  1|    James|          |   Smith|36636|     M|        1000|     Sales|2020|
|  2|    Maria|      Anne|   Jones|39192|     F|        4000|Operations|2020|
|  3|  Michael|      Rose|        |40288|     M|        2000|Operations|2020|
|  4|   Robert|          |Williams|42114|     M|        3000|     Sales|2020|
|  5|      Ria|      Anne|   Jones|60000|     F|        7000|Operations|2020|
+---+---------+----------+--------+-----+------+------------+----------+----+



#monotonically_increasing_id and row_number

Now, if we dont have a sortable data (on any column), then we can opt for a combination of **row_number** with **monotonically_increasing_id**.



In [18]:

from pyspark.sql import Window
from pyspark.sql.functions import row_number
df = df1.withColumn("counter", monotonically_increasing_id())
w = Window.orderBy("counter")

# Use row number with the window specification
df_index = df.withColumn("index", row_number().over(w))
df_index = df_index.drop("counter")
df_index.show(truncate=False)


+---------+----------+--------+-----+------+------------+----------+----+-----+
|firstname|middlename|lastname|id   |gender|annualsalary|work      |year|index|
+---------+----------+--------+-----+------+------------+----------+----+-----+
|James    |          |Smith   |36636|M     |1000        |Sales     |2020|1    |
|Michael  |Rose      |        |40288|M     |2000        |Operations|2020|2    |
|Robert   |          |Williams|42114|M     |3000        |Sales     |2020|3    |
|Maria    |Anne      |Jones   |39192|F     |4000        |Operations|2020|4    |
|Ria      |Anne      |Jones   |60000|F     |7000        |Operations|2020|5    |
+---------+----------+--------+-----+------+------------+----------+----+-----+



#monotonically_increasing_id and coalesce

 coalesce(1) puts the Dataframe in one partition, and so have monotonically increasing and successive index column.

 However coalesce(1) also moves the entire data into a single partition, the risk of memory issues still exists. 

In [19]:
df1.coalesce(1).withColumn("idx11", monotonically_increasing_id()).show()


+---------+----------+--------+-----+------+------------+----------+----+-----+
|firstname|middlename|lastname|   id|gender|annualsalary|      work|year|idx11|
+---------+----------+--------+-----+------+------------+----------+----+-----+
|    James|          |   Smith|36636|     M|        1000|     Sales|2020|    0|
|  Michael|      Rose|        |40288|     M|        2000|Operations|2020|    1|
|   Robert|          |Williams|42114|     M|        3000|     Sales|2020|    2|
|    Maria|      Anne|   Jones|39192|     F|        4000|Operations|2020|    3|
|      Ria|      Anne|   Jones|60000|     F|        7000|Operations|2020|    4|
+---------+----------+--------+-----+------+------------+----------+----+-----+

